# Week 05


## After-class report

### Reading

- [Chapter 10](https://jeffe.cs.illinois.edu/teaching/algorithms/book/10-maxflow.pdf) on _Maximum Flows and Minimum Cuts_ from Jeff Erickson's book.
- [Ford and Fulkerson's original paper](./OER/ford-max_flow-1956.pdf) from 1956.


### Problem statement

Given a directed graph whose edges represent flow capacities between its vertices, what is the maximum flow that can move between a source and a destination vertex? For example in the graph **(a)** below, what is the maximum flow that we can move from vertex 0 to vertex 3?


![](https://raw.githubusercontent.com/lgreco/images/refs/heads/main/graphs/ford_fulkerson_ab.png)


### Strategy

The answer to the problem is the Ford Fulkerson method that computes the maximum flow between two vertices.

$$
\begin{align*}
& \textbf{maximum flow in graph}\ {\color{blue}G}\ \textbf{between vertices}\ s, d: \\
& \qquad {\color{purple}R} \leftarrow \text{residual graph of}\ {\color{blue}G} \\
& \qquad f_{\max} \leftarrow 0 \\
& \qquad \textbf{while}\ \text{there is an augmenting path in}\ {\color{purple}R:}\\
& \qquad \quad f_\text{path} \leftarrow \text{minimum capacity across augmenting path} \\
& \qquad \quad f_{\max} \leftarrow f_{\max}+f_\text{path} \\
& \qquad \quad \text{saturate augmenting path}\\
& \qquad \textbf{return}\ f_{\max} \\
\end{align*}
$$


### Residual graph

The residual graph ${\color{purple}R}$, shown in **(b)** above is a copy of the input graph ${\color{blue}G}$. The algorithm requires adjusting and even eliminating edges, and we can't mutate the input graph. All the mutations need to be done, safely, on a replica. That replica is called the **residual graph.** Initially it's an exact copy of the input graph.

If  ${\color{blue}G}=(V,{\color{blue}E})$, then initially ${\color{purple}R}=(V,{\color{blue}E})$ as well. When the process ends, ${\color{purple}R}=(V,{\color{purple}E_\text{residual}})$ such that ${\color{purple}E_\text{residual}}\subseteq{\color{blue}E}$.

### Augmenting path

An augmenting path is _any path_ between the source and the destination vertices in the residual graph. Figure **(c)** highlights an augmenting path.

![](https://github.com/lgreco/images/blob/main/graphs/ford_fulkerson_c.png?raw=true)


### Finding $f_\text{path}$

An augmenting path can carry as much flow as we can send through its weakest edge. For example, the path in figure **(c)** above can carry 2 units of flow. Because its weakest edge has a capacity of 2.

In general, an augmentic path $s\rightsquigarrow d$ is in the form

$$ s\rightarrow p_1 \rightarrow p_2 \rightarrow\ldots\rightarrow d$$

where $p_i$ are the vertices along the edges of the augmenting path.

If we represent an augmenting path as an array or list, `path = [s, p1, p2, ..., d]`, finding its maximum possible flow $f_\text{path}$ becomes a simple linear search.

```python
f_path = float('inf')
i = 0
while i < len(path) - 1:
    u = path[i]
    v = path[i + 1]
    w = G[u][v]
    if w < f_path:
        f_path = w
    i += 1
```

At the end of the loop above, `f_path` is the smallest capacity along the edges of the augmenting path. This capacity is also the maximum flow we can send down that path.


### Saturate the augmenting path

Looking at the augmenting path of our example again, it's clear that its maximum _possible_ flow is $f_\text{path}=2$.

![](https://github.com/lgreco/images/blob/main/graphs/ford_fulkerson_c.png?raw=true)

This path can be saturated by sending that much flow along it. The saturation is depicted below.

![](https://github.com/lgreco/images/blob/main/graphs/ford_fulkerson_d.png?raw=true)

What happens here is that the effective capacity along the edges of the augmenting path $0\rightarrow 1\rightarrow 2\rightarrow 3$ is reduced by the flow we are pushing. At the same time, that flow is represented as reverse edges $3\rightarrow 2\rightarrow 1\rightarrow 0$. Because the flow we are pushing equals to the capacity of the weakest edge ($2\rightarrow 3$), that edge is effectively cancelled. The augmenting path $0\rightarrow 1{\color{red}\rightarrow} 2\rightarrow 3$ is eliminated.

There is one more augmenting path in the graph, shown in figure **(e)** below.

![](https://github.com/lgreco/images/blob/main/graphs/ford_fulkerson_ef.png?raw=true)

After saturating that path, we lose the edge $4{\color{red}\rightarrow}2$. This augmenting path is eliminated as well (figure **(f)** above). Effectively, the graph is now partitioned into two components as shown in figure **(g)** below.

![](https://github.com/lgreco/images/blob/main/graphs/ford_fulkerson_g.png?raw=true)


One component is the set $\mathcal S$ of the vertices that can be reached from the source vertex $s$. The other component is the set $\mathcal D$ of vertices that cannot be reached from $s$. The sets are mutually exclusive, $\mathcal S\cap\mathcal D = \varnothing$ but also $\mathcal S\cup\mathcal D = V$, where $V$ is the set of vertices of the graph.


## Finding a path, any path

Everything above depends on our ability to find a path between two vertices in a graph. Path finding is a standard graph algorithm, given below in Python.

This algorithm performs a **depth-first search (DFS)** starting at the `source` vertex and explores outward along directed edges until it either reaches the `destination` or exhausts all reachable vertices. It uses a **stack** to control exploration: the most recently discovered vertex is processed next, which causes the search to follow one path “deep” into the graph before backtracking.

A `visited` list prevents revisiting vertices, and a `parent` list records the predecessor from which each vertex was first discovered. When the destination vertex is encountered, the algorithm reconstructs the path by following the recorded parent links backward from `destination` to `source`, then reversing that sequence.


In [ ]:
def find_path(G: list[list[int]], source: int, destination: int) -> list[int] | None:
    """
    Returns any path from source to destination in a directed graph
    represented as an adjacency matrix where 0 means no edge.
    If no path exists, returns None.
    """
    # Number of vertices in the graph
    n: int = len(G)
    # visited[v] is True once vertex v has been discovered
    visited: list[bool] = [False for _ in range(n)]
    # parent[v] stores the vertex from which v was first reached
    # This will allow us to reconstruct the path later
    parent: list[int | None] = [None for _ in range(n)]
    # Stack used to implement iterative DFS
    stack: list[int] = [source]
    # Mark the start vertex as visited immediately
    visited[source] = True
    # Flag to indicate whether we have found the goal
    found: bool = False
    # Continue searching while there are vertices to explore
    # and the goal has not yet been found
    while len(stack) > 0 and not found:
        # Take the most recently added vertex (DFS behavior)
        u: int = stack.pop()
        # If we reached the goal, stop exploring
        if u == destination:
            found = True
        else:
            # Examine all possible outgoing edges u -> v
            v: int = 0
            while v < n:
                # If there is an edge from u to v
                if G[u][v] != 0:
                    # If v has not yet been visited, discover it
                    if not visited[v]:
                        visited[v] = True
                        parent[v] = u  # record how we reached v
                        stack.append(v)  # explore v later
                v += 1
    # Prepare the result (None unless we found a path)
    path: list[int] | None = None
    # If goal was found, reconstruct the path
    if found:
        path: list[int] = []
        current: int | None = destination
        # Follow parent links backward from goal to start
        while current is not None:
            path.append(current)
            current = parent[current]
        # Reverse to obtain start -> goal order
        path.reverse()
        path = path
    return path

The code above can be tested with the simple graph we used in this notebook:


In [8]:
# fmt:off

graph = [
    # 0   1   2   3   4
    [ 0, 10,  0,  0,  4], # 0
    [ 0,  0,  2,  0,  0], # 1
    [ 0,  0,  0, 10,  0], # 2
    [ 0,  0,  0,  0,  0], # 3
    [ 0,  0,  2,  0,  0]  # 4
]

source_vertex = 0
destination_vertex = 3
print(find_path(graph, source_vertex, destination_vertex))

[0, 4, 2, 3]



---

### Part 1

Using method `find_path`, we can put together some code to find the maximum flow across a graph whose edges represent flow capacities. The outline of the code is given below and **your job is to complete it.**  "All" you have to do is to flesh out what's going on in the `while` loop below.


In [9]:
from copy import deepcopy  # for deep copy of input graph 

def max_flow(G:list[list[int]], source:int, destination:int):
    # Shortcut to size of graph
    n = len(G)
    # Initialize return value
    f_max = 0
    # Create residual graph. Need deep copy to avoid
    # mutating the input graph G
    R = deepcopy(G)
    # Find an initial augmenting path in R
    augmenting = find_path(R, source, destination)
    # While there is an augmenting path
    while augmenting is not None:
        # f_path = smallest edge capacity along the augmenting path
        f_path = float('inf')
        i = 0
        while i < len(augmenting) - 1:
            u = augmenting[i]
            v = augmenting[i + 1]
            w = R[u][v]
            if w < f_path:
                f_path = w
            i += 1
        # Add the path's capacity to the graph's max flow
        f_max += f_path
        # Reduce capacity of forward edges and add/increase reverse edges
        i = 0
        while i < len(augmenting) - 1:
            u = augmenting[i]
            v = augmenting[i + 1]
            # Reduce forward edge capacity by f_path
            R[u][v] -= f_path
            # If capacity reaches 0, remove the edge
            if R[u][v] == 0:
                R[u][v] = 0  # already 0, edge effectively removed
            # Add reverse edge with f_path (or increase existing reverse capacity)
            R[v][u] += f_path
            i += 1
        # Find the next augmenting path in R
        augmenting = find_path(R, source, destination)
    # Done - return max flow and residual graph (needed for Part 2)
    return f_max, R


# Test with the example graph
graph = [
    # 0   1   2   3   4
    [ 0, 10,  0,  0,  4], # 0
    [ 0,  0,  2,  0,  0], # 1
    [ 0,  0,  0, 10,  0], # 2
    [ 0,  0,  0,  0,  0], # 3
    [ 0,  0,  2,  0,  0]  # 4
]

flow, R = max_flow(graph, 0, 3)
print(f'Maximum flow from 0 to 3: {flow}')  # Expected: 4

IndentationError: expected an indented block after 'while' statement on line 14 (474178384.py, line 21)


---

### Part 2

At the end of the process in method `max_flow` from part 1, the residual graph is separated into two components, as we already saw: $\mathcal S$, the set of vertices reachable from the source vertex $s$ and $\mathcal D$, those that are not. We can compute $\mathcal S$ quite easily. After that, finding $\mathcal D$ is also easy: $\mathcal D = V\setminus\mathcal S$, the difference between sets $V$ and $\mathcal S$.

The edges, in graph $G$ that connect vertices between $\mathcal S$ and $\mathcal D$ in graph $R$ are called the **minimum cut:** these are the edges that we need to distrupt, to stop the maximum from from $s$ to $d$ in the graph.

The set of vertices reachable from another vertex can be found with:

In [ ]:
def reach(graph:list[list[int]], s:int) -> list[int]:
    # Shortcuts
    n = len(graph)
    no_edge = graph[0][0]
    reachable = []
    explore_next = [s]
    while explore_next:
        u = explore_next.pop()
        if u not in reachable:
            reachable.append(u)
            for v in range(n):
                if graph[u][v] != no_edge:
                    if v not in explore_next:
                        explore_next.append(v)
    return reachable

With this method available, we can obtain the edges of the minimum cut $ \mathcal M$ using the following process.

$$
\begin{align*}
& \qquad \textbf{find min cut}({\color{blue}G}=(V,{\color{blue}E}),\ {\color{purple}R}=(V,{\color{purple}E_\text{residual}}),\ s): \\
& \qquad \mathcal M \leftarrow\varnothing \\
& \qquad \mathcal S \leftarrow \texttt{reach(R, s)}\\
& \qquad \mathcal D \leftarrow V\setminus\mathcal S\\
& \qquad \forall u\in\mathcal S:\\
& \qquad \quad \forall (u,v)\in {\color{blue}E}: \\
& \qquad \quad \quad \textbf{if}\ v\in \mathcal D:\\
& \qquad \quad \quad \quad \mathcal M \leftarrow \mathcal M \cup \{(u,v)\} \\
& \qquad \textbf{return}\ \mathcal M
\end{align*}
$$

Modify method `max_flow` to return the the residual graph `R` as well as `f_max`. Then use `R` to derive the minimum cut of graph `G` using the technique shown above.

The operation $\mathcal D \leftarrow V\setminus\mathcal S$ is also straigthforward to implement:

```python
S = reach(R, s)
D = []
for i in in range(n):
    if i not in S:
        D.append(i)
```

There is even a more efficient way to do this:

```python
S = reach(R, s)
reachable = [False] * n
for i in range(n):
    reachable[i] = (i in S)
```

In [ ]:
def min_cut(G: list[list[int]], source: int, destination: int):
    """
    Finds the minimum cut of graph G between source and destination.
    Returns the max flow value and list of edges (u, v) forming the minimum cut.
    """
    n = len(G)
    # Run max_flow to get the final residual graph R
    f_max, R = max_flow(G, source, destination)
    # S = set of vertices reachable from source in residual graph
    S = reach(R, source)
    # Mark which vertices are reachable (in S) vs unreachable (in D)
    reachable = [False] * n
    for i in range(n):
        reachable[i] = (i in S)
    # M = edges in original graph G that cross from S to D
    M = []
    for u in S:
        for v in range(n):
            if G[u][v] != 0 and not reachable[v]:
                M.append((u, v))
    return f_max, M


# Test with the example graph
graph = [
    # 0   1   2   3   4
    [ 0, 10,  0,  0,  4], # 0
    [ 0,  0,  2,  0,  0], # 1
    [ 0,  0,  0, 10,  0], # 2
    [ 0,  0,  0,  0,  0], # 3
    [ 0,  0,  2,  0,  0]  # 4
]

f_max, cut_edges = min_cut(graph, 0, 3)
print(f'Maximum flow: {f_max}')           # Expected: 4
print(f'Minimum cut edges: {cut_edges}')  # Expected: edges (1,2) and (4,2)